### Portfolio Management System

지금까지 배운 내용을 바탕으로 개인의 투자를 도와주는 Management System을 LLM 모델을 통해 만들어 보도록 하자. 디스플레이를 위해 앱 구동은 streamlit library를 사용한다.

### 1. Agent Modeling

금융 데이터를 input으로 하여 LLM으로부터 분석을 의뢰한다.

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [2]:
import yfinance as yf

ASSET = 'TSLA'

data = yf.download(
    ASSET, start='2000-01-01', 
    interval='1d', 
    progress=False, 
    auto_adjust=True, 
    multi_level_index=False 
).loc['2000':]

#### 1.1 Technical Indicator Analyst

Technical Indicator만을 전문적으로 분석하여 현재 상황이 Bullish, Bearish, Neutral 중 어느 상황에 가까운지 판단하는 에이전트이다. pandas.DataFrame 자체를 input으로 이용한다.

In [3]:
from utils import *

momentum = MomentumIndicator(data, resample='1d')
contrarian = ContrarianIndicator(data, resample='1d')

In [4]:
# Simple moving average
sma_5 = momentum.simple_moving_average(5)
sma_20 = momentum.simple_moving_average(20)
sma_golden_cross = ((sma_5 >= sma_20) & (sma_5.shift(1) < sma_20.shift(1))).astype(int)
sma_dead_cross = ((sma_5 < sma_20) & (sma_5.shift(1) >= sma_20.shift(1))).astype(int)

# True Strength Index
tsi_25_13 = momentum.true_strength_index(13, 25)
tsi_up = ((tsi_25_13 >= 0) & (tsi_25_13.shift(1) < 0)).astype(int)
tsi_down = ((tsi_25_13 < 0) & (tsi_25_13.shift(1) >= 0)).astype(int)

# ADL
adl14 = momentum.average_daily_range(14)
adl50 = momentum.average_daily_range(50)
adl_up = ((adl14 >= adl50) & (adl14.shift(1) < adl50.shift(1))).astype(int)
adl_down = ((adl14 < adl50) & (adl14.shift(1) >= adl50.shift(1))).astype(int)

# ADR
adr20 = momentum.average_daily_range(20)
adr50 = momentum.average_daily_range(50)
adr_up = ((adr20 >= adr50) & (adr20.shift(1) < adr50.shift(1))).astype(int)
adr_down = ((adr20 < adr50) & (adr20.shift(1) >= adr50.shift(1))).astype(int)

# Aroon
aroon = momentum.aroon_indicator(14)
aroon_up = ((aroon['Aroon(14) up'] >= aroon['Aroon(14) down']) & (
            aroon['Aroon(14) up'].shift(1) < aroon['Aroon(14) down'].shift(1))).astype(int)
aroon_down = ((aroon['Aroon(14) up'] < aroon['Aroon(14) down']) & (
        aroon['Aroon(14) up'].shift(1) >= aroon['Aroon(14) down'].shift(1))).astype(int)

# RSI
rsi = contrarian.relative_strength_index(20)
rsi_up = ((rsi <= 70) & (rsi.shift(1) > 70)).astype(int)
rsi_down = ((rsi >= 30) & (rsi.shift(1) < 30)).astype(int)

# BB
bb = contrarian.bollinger_band(2, 20)
bb_up = ((data['Close'] < bb['BB_UP(2)']) & (data['Close'].shift(1) >= bb['BB_UP(2)'].shift(1))).astype(int)
bb_down = ((data['Close'] >= bb['BB_DOWN(2)']) & (data['Close'].shift(1) < bb['BB_DOWN(2)'].shift(1))).astype(int)

# DEMARKER
demark = contrarian.demarker_indicator(20)
demark_up = ((demark < 0.7) & (demark.shift(1) >= 0.7)).astype(int)
demark_down = ((demark >= 0.3) & (demark.shift(1) < 0.3)).astype(int)

# psycological line
psycological_line = contrarian.psycological_line(20)
pl_up = ((psycological_line < 0.7) & (psycological_line.shift(1) >= 0.7)).astype(int)
pl_down = ((psycological_line >= 0.3) & (psycological_line.shift(1) < 0.3)).astype(int)

#concatenate
res = pd.concat(
    [sma_golden_cross, sma_dead_cross,tsi_up, tsi_down, adl_up, adl_down, adr_up, adr_down, aroon_up, aroon_down, rsi_up, rsi_down,bb_up, bb_down, demark_up, demark_down,
     pl_up, pl_down], axis = 1
)
res.columns = [
    'SMA(5,20) Golden', 'SMA(5,20) Dead', 
    'TSI(13,25) Golden', 'TSI(13,25) Dead',
    'ADL(14,50) Golden', 'ADL(14,50) Dead',
    'ADR(20,50) Golden', 'ADR(20,50) Dead',
    'Aroon(14) Golden', 'Aroon(14) Dead',
    'RSI(30,70) up','RSI(30,70) down',
    'Bollinger Band(2, 20) up','Bollinger Band(2, 20) down',
    'Demark(30,70) up','Demark(30,70) down',
    'Psy (30, 70) up','Psy (30, 70) down'
]

In [5]:
res.tail()

,"SMA(5,20) Golden","SMA(5,20) Dead","TSI(13,25) Golden","TSI(13,25) Dead","ADL(14,50) Golden","ADL(14,50) Dead","ADR(20,50) Golden","ADR(20,50) Dead",Aroon(14) Golden,Aroon(14) Dead,"RSI(30,70) up","RSI(30,70) down","Bollinger Band(2, 20) up","Bollinger Band(2, 20) down","Demark(30,70) up","Demark(30,70) down","Psy (30, 70) up","Psy (30, 70) down"
Date,,,,,,,,,,,,,,,,,,
2025-02-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-02-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-02-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-02-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-02-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableSequence
import warnings
warnings.filterwarnings('ignore')

def technical_agent(signals):
    """
    OpenAI GPT API를 사용하여 0과 1로 구성된 기술적 지표 신호를 해석하고
    시장 상황을 Bullish, Bearish, Neutral 중 하나로 판단하여 매매 조언을 제공하는 함수입니다.
    :param signals: signal이 기록된 pd.DataFrame형태의 
    """
    llm = ChatOpenAI(
        model = "gpt-4o-mini", 
        temperature = 0.5, 
        openai_api_key = api_key
    )
    
    prompt = """
    다음은 최근 금융 시장의 기술적 지표 신호(0 또는 1) 기록입니다.
    1은 해당 신호가 발생했음을 의미하고, 0은 신호가 발생하지 않았음을 의미합니다.

    최근 40개 데이터:
    {signals}

    판단 기준:
    - 매수 신호가 다수 발생하면 Bullish (강세)로 판단.
    - 매도 신호가 다수 발생하면 Bearish (약세)로 판단.
    - 신호가 혼재되어 있거나 강한 추세가 없으면 Neutral (중립)로 판단.
    - 현재 상태에 따라 투자자에게 Buy(매수), Sell(매도), Hold(관망) 조언을 제공.

    시장 상황을 분석하고, 현재 상태를 Bullish, Bearish, Neutral 중 하나로 판단한 후
    투자자에게 적절한 조언을 한 줄로 제공하세요.
    """

    # OpenAI GPT-4 호출
    prompt = PromptTemplate(
        template=prompt,
        input_variables=["signals"]
    )

    # RunnableSequence를 사용하여 체인 구성
    chain = RunnableSequence(
        {
            "signals": RunnablePassthrough()
        }
        | prompt
        | llm
        | (lambda x: x.content)
    )

    latest_signals = signals.to_string(index=False)

    # 시장 분석 실행
    result = chain.invoke({"signals": latest_signals})
    
    return result

In [9]:
tech_resp = technical_agent(res.tail(40))

In [10]:
print(tech_resp)

주어진 신호 데이터에 따르면, 매수 신호는 거의 없고 매도 신호도 적은 편이며, 전반적으로 신호가 혼재되어 있습니다. 따라서 현재 상태는 Neutral (중립)으로 판단됩니다. 투자자에게는 "Hold(관망)"을 권장합니다.


### 1.2 Fundamental Analyst

yahoo finance로부터 기업의 재무상황을 불러올 수 있다. 이를 LLM에 넣고 판단을 해주는 에이전트를 생성한다.

**Fundamental Data Import**

재무 데이터만을 가져오는 함수를 먼저 지정한다

In [11]:
def get_fundamental_data(ticker):
    """
    Yahoo Finance API를 사용하여 해당 종목의 기본적 분석(Fundamental Analysis) 데이터를 가져오는 함수
    :param ticker: 종목 코드 (예: "AAPL", "TSLA")
    :return: 해당 종목의 재무 지표가 포함된 pd.DataFrame
    """
    stock = yf.Ticker(ticker)
    
    # 종목의 주요 재무 지표 가져오기
    try:
        info = stock.info
        data = {
            "Ticker": ticker,
            "Company": info.get("longName", "N/A"), # 회사 이름 
            "Sector": info.get("sector", "N/A"), # 섹터 이름, GICS기준
            "Market Cap": info.get("marketCap", "N/A"), # 시가총액
            "PER": info.get("trailingPE", "N/A"), # Price to Earning ratio
            "Forward PER": info.get("forwardPE", "N/A"), # forward PER
            "PBR": info.get("priceToBook", "N/A"), # Price to Book ratio
            "EV/EBITDA": info.get("enterpriseToEbitda", "N/A"), # EV/EBITDA
            "52 Week High": info.get("fiftyTwoWeekHigh", "N/A"), # 52주 최고가
            "52 Week Low": info.get("fiftyTwoWeekLow", "N/A"), # 52주 최저가
            "Dividend Yield": info.get("dividendYield", "N/A"), # 배당수익률
            "Recommendation": info.get("recommendationKey", "N/A"), 
        }
        return pd.DataFrame([data])
    
    except Exception as e:
        print(f"data download error: {e}")
        return None

In [12]:
def fundamental_agent(ticker):
    """
    OpenAI GPT API를 사용하여 Yahoo Finance에서 불러온 재무 데이터를 기반으로
    주식이 고평가(Bearish)인지, 저평가(Bullish)인지 분석하고, 매매 조언을 제공하는 함수.
    
    :param ticker: 주식 종목 코드 (예: "AAPL", "GOOGL")
    :return: LLM 분석 결과 (Bullish, Bearish, Neutral 및 매매 조언)
    """
    # Yahoo Finance에서 재무 데이터 가져오기
    fund_data = get_fundamental_data(ticker)
    
    if fund_data is None:
        return {"signal": "Error", "message": "재무 데이터 불러오기 실패"}
    
    llm = ChatOpenAI(
        model = "gpt-4o-mini", 
        temperature = 0.5, 
        openai_api_key = api_key
    )

    # LLM 프롬프트 템플릿 설정
    prompt = """
    다음은 {ticker}({company})의 최신 펀더멘털(재무) 데이터입니다:
    {fundamental_data}

    판단 기준:
    - PER, PBR, EV/EBITDA가 업종 평균보다 낮으면 저평가(Bullish)로 판단.
    - 반대로 PER, PBR, EV/EBITDA가 높으면 고평가(Bearish)로 판단.
    - 업종 평균과 비슷한 수준이면 중립(Neutral)로 판단.
    - 배당률이 높거나 성장성이 좋은 경우도 고려하여 판단.
    - 현재 상태에 따라 투자자에게 Buy(매수), Sell(매도), Hold(관망) 조언을 제공.

    재무 상황을 분석하고, 현재 상태를 Bullish, Bearish, Neutral 중 하나로 판단한 후
    투자자에게 적절한 조언을 한 줄로 제공하세요.
    """

    prompt = PromptTemplate(
        template=prompt,
        input_variables=["ticker", "company", "fundamental_data"]
    )

    # RunnableSequence를 사용하여 체인 구성
    chain = RunnableSequence(
        {
            "ticker": RunnablePassthrough(),
            "company": RunnablePassthrough(),
            "fundamental_data": RunnablePassthrough()
        }
        | prompt
        | llm
        | (lambda x: x.content)
    )

    # LLM 실행
    result = chain.invoke({
        "ticker": fund_data["Ticker"][0],
        "company": fund_data["Company"][0],
        "fundamental_data": fund_data.to_string(index=False)
    })

    return result

In [13]:
fund_resp = fundamental_agent("TSLA")

In [14]:
fund_resp

'Tesla, Inc. (TSLA)의 PER(142.92), PBR(12.92), EV/EBITDA(70.64)는 업종 평균보다 높은 수준으로, 따라서 고평가(Bearish)로 판단됩니다. 현재 상태에 따라 투자자에게 "Sell(매도)" 조언을 제공합니다.'

### 1.3 Sentiment Analyst

투자 의사결정에는 Technical지표, 재무 지표 뿐만 아니라 시장의 Sentiment 또한 중요하다. google news로부터 기사를 수집한 뒤, 이를 기반으로 매도, 매수 조언을 하는 에이전트를 생성한다.

**news parser**

market sentiment를 구하기 위해 우선 뉴스를 스크랩하는 함수를 정의해야 한다. 기존에 사용되었던 `search_from_google`함수를 사용하자.

In [15]:
from bs4 import BeautifulSoup
import requests
import urllib
import re
import pandas as pd
from newspaper import Article
from selenium import webdriver

def search_from_google(asset:str, page_nums:int) -> pd.DataFrame :
    '''
    google news로부터 검색을 한 뒤, selenium을 통해 뉴스 데이터들을 가져옵니다.
    :param asset: 검색할 자산
    :param page_nums: 뉴스를 검색할 총 페이지의 수
    :return: news data가 들어있는 DataFrame
    '''
    keyword = f'{asset} buying reason'
    news_df = pd.DataFrame()
    
    # selenium headless mode
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    
    for page_num in range(0, page_nums-1):
        # google news crawling
        url = f'https://www.google.com/search?q={keyword}&sca_esv=1814fa2a4600643d&tbas=0&tbs=qdr:m&tbm=nws&ei=rE3pZeLxNeHX1e8PpdOcMA&start={page_num}&sa=N&ved=2ahUKEwji9-zrsuGEAxXha_UHHaUpBwYQ8tMDegQIBBAE&biw=2560&bih=1313&dpr=1'
        req = requests.get(url)
        content = req.content
        soup = BeautifulSoup(content, 'html.parser')
    
        # last page check
        if soup.select('div.BNeawe.vvjwJb') == []: break
    
        title_list = [t.text for t in soup.select('div.BNeawe.vvjwJb')]  # title
        url_list = []
    
        # url
        for u in soup.select('a'):
            for t in title_list:
                if t in u.text:
                    temp_url = urllib.parse.unquote(u['href'])
                    temp_url = re.findall('http\S+&sa',temp_url)[0][:-3]
                    url_list.append(temp_url)
    
        # article
        for ind, news_url in enumerate(url_list):
            try:
                article = Article(url=news_url)
                article.download()
                article.parse()    
                news_article = article.text
            except:  # ssl error
                driver.get(news_url)
                article.download(input_html=driver.page_source)
                article.parse()
                news_article = article.text
    
            news_df = pd.concat([news_df, pd.DataFrame([[title_list[ind], news_article, news_url]])])
    
        news_df[0] = news_df[0].apply(lambda x: re.sub('\s+',' ',x))
        news_df = news_df.reset_index(drop=True)
    
    news_df.columns = ['Title','Contents','URL']
    
    return news_df

In [16]:
news_df = search_from_google(ASSET, page_nums=4)

In [18]:
news_df['Contents']

0     BitMart's Strategy & Growth | FMTalks with Kse...
1     Access to this page has been denied because we...
2           Please enable JS and disable any ad blocker
3     Tesla (TSLA 3.91%) stock has been in freefall,...
4     Stocks ended the week a little lower with the ...
5     Ross Gerber was an early investor in Tesla, bu...
6     You can also send us your feedback:\n\nI’m a b...
7                                                      
8     Switch the Market flag\n\nOpen the menu and sw...
9     Elon Musk may be President Trump's first buddy...
10    Access to this page has been denied because we...
11                                                     
12    Tesla (TSLA 3.91%) stock has been in freefall,...
13    teslamotorsclub.com\n\nVerifying you are human...
14    Stocks ended the week a little lower with the ...
15    Ross Gerber was an early investor in Tesla, bu...
16    You can also send us your feedback:\n\nI’m a b...
17                                              

In [19]:
def sentiment_agent(news_df):
    """
    OpenAI GPT API를 사용하여 뉴스 기사의 제목과 본문을 분석하고,
    해당 기업에 대한 뉴스가 긍정적인지(Bullish), 부정적인지(Bearish), 중립적인지(Neutral) 판단하여
    투자자에게 조언을 제공하는 함수.

    :param news_df: 뉴스 데이터프레임 (컬럼: ['Title', 'Contents', 'URL'])
    :return: LLM 분석 결과 (Bullish, Bearish, Neutral 및 매매 조언)
    """
    if news_df is None or news_df.empty:
        return {"signal": "Error", "message": "뉴스 데이터가 없습니다!"}

    news_df["Full_Text"] = news_df["Title"] + " " + news_df["Contents"]
    latest_news = news_df["Full_Text"].tolist()[:10]  # 최신 뉴스만을 사용
    
    llm = ChatOpenAI(
        model = "gpt-4o-mini", 
        temperature = 0.5, 
        openai_api_key = api_key
    )

    # LLM 프롬프트 템플릿 설정
    template = """
    다음은 기업과 관련된 최신 뉴스 기사들입니다.  
    각 뉴스는 해당 기업에 대한 투자 심리를 반영할 수 있으며, 감성 분석을 통해 시장 분위기를 평가하세요.

    최근 5개 뉴스 기사:
    {news}

    판단 기준:
    - 긍정적인 뉴스 기사가 다수라면 Bullish (강세)로 판단.
    - 부정적인 뉴스 기사가 다수라면 Bearish (약세)로 판단.
    - 긍정과 부정이 혼재되어 있거나, 특별한 정보가 없다면 Neutral (중립)로 판단.
    - 시장 심리를 반영하여 투자자에게 Buy(매수), Sell(매도), Hold(관망) 조언을 제공.

    현재 상태를 Bullish, Bearish, Neutral 중 하나로 판단한 후
    투자자에게 적절한 조언을 한 줄로 제공하세요.
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["news"]
    )

    # RunnableSequence를 사용하여 체인 구성
    chain = RunnableSequence(
        {
            "news": RunnablePassthrough()
        }
        | prompt
        | llm
        | (lambda x: x.content)
    )

    # LLM 실행
    result = chain.invoke({
        "news": "\n\n".join(latest_news)  # 뉴스 본문 결합
    })

    return result

In [20]:
sent_resp = sentiment_agent(news_df)

In [21]:
sent_resp

'현재 상태는 Bearish (약세)입니다. \n\n투자자에게는 "Tesla 주식은 현재 하락세에 있으므로 매도하는 것이 좋습니다."라고 조언합니다.'

### 1.4 Forecaster

가격 데이터를 활용하여, 기본적인 통계 모형을 활용해 그 다음날의 수익률을 예측하는 Agent를 생성한다.

In [22]:
from statsmodels.tsa.arima.model import ARIMA

def get_stock_data(ticker):
    """
    Yahoo Finance에서 종가 데이터를 가져오는 함수
    :param ticker: 종목 코드 (예: "AAPL", "TSLA")
    :return: 종가 및 수익률 데이터가 포함된 pd.DataFrame
    """
    
    data = yf.download(
        ticker,
        start = '2024-01-01',
        progress=False,
        multi_level_index=False,
        interval='1d',
        auto_adjust=True
    )
    
    if data.empty:
        return None

    data = data['Close'].pct_change().dropna()
    
    return data

def estimate_arma_model(returns):
    """
    주어진 수익률 데이터로 ARMA(1,1) 모형을 추정하고 1-step ahead forecast 값을 반환하는 함수
    :param returns: 수익률 데이터 (pd.Series)
    :return: 1-step ahead 예측값
    """
    try:
        # ARMA(1,1) 모형 적합
        model = ARIMA(returns, order=(3,0,1))  # (AR=1, MA=1)
        model_fit = model.fit()
        
        # 1-step ahead 예측값 추출
        forecast = model_fit.forecast(steps=1)
        
        return forecast

    except Exception as e:
        print(f"ARMA 모형 추정 실패: {e}")
        return None

In [23]:
import warnings
warnings.filterwarnings("ignore")

def forecaster_agent(ticker):
    """
    OpenAI GPT API를 사용하여 ARMA(1,1) 모형을 기반으로 시장 예측을 수행하고,
    예측된 값이 상승이면 Bullish, 하락이면 Bearish, 중립이면 Neutral로 분류하여 투자 조언을 제공하는 함수.

    :param ticker: 주식 종목 코드 (예: "AAPL", "GOOGL")
    :return: LLM 분석 결과 (Bullish, Bearish, Neutral 및 매매 조언)
    """
    # Yahoo Finance에서 주가 데이터 가져오기
    stock_returns = get_stock_data(ticker)
    
    if stock_returns is None:
        return {"signal": "Error", "message": "주가 데이터 불러오기 실패"}
    
    llm = ChatOpenAI(
        model = "gpt-4o-mini", 
        temperature = 0.5, 
        openai_api_key = api_key
    )

    # ARMA(1,1) 모형 추정 및 1-step ahead 예측
    forecast = estimate_arma_model(stock_returns)
    
    if forecast is None:
        return {"signal": "Error", "message": "❌ ARMA 모델 예측 실패!"}

    # LLM 프롬프트 템플릿 설정
    template = """
    다음은 {ticker}의 최근 1년간 종가 데이터를 기반으로 한 ARMA(1,1) 모형의 예측 결과입니다.

    1-step ahead 예측값: {forecast}

    판단 기준:
    - 예측값이 양수(>0)이면 Bullish (강세)로 판단.
    - 예측값이 음수(<0)이면 Bearish (약세)로 판단.
    - 예측값이 0에 가까우면 Neutral (중립)로 판단.
    - 현재 상태에 따라 투자자에게 Buy(매수), Sell(매도), Hold(관망) 조언을 제공.

    현재 상태를 Bullish, Bearish, Neutral 중 하나로 판단한 후
    투자자에게 적절한 조언을 한 줄로 제공하세요.
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["ticker", "forecast"]
    )

    # RunnableSequence를 사용하여 체인 구성
    chain = RunnableSequence(
        {
            "ticker": RunnablePassthrough(),
            "forecast": RunnablePassthrough()
        }
        | prompt
        | llm
        | (lambda x: x.content)
    )

    # LLM 실행
    result = chain.invoke({
        "ticker": ticker,
        "forecast": forecast
    })

    return result

In [24]:
forecast_resp = forecaster_agent(ASSET)

In [25]:
forecast_resp

'예측값이 -0.000227로 음수이므로 Bearish (약세)로 판단되며, 투자자에게 Sell(매도) 조언을 드립니다.'

- META Labeling 
- Agent
    - 워렌버핏 에이전트 : 최근 데이터를 인풋으로 받아서, 현금 보유비중을 얼마나 두어야 하는지 알려주는 에이전트

### 2. Manager

agent들로부터 나온 결과를 취합받아 최종적으로 조언을 해주는 포트폴리오 매니저의 역할이 필요하다. 이는 헤지펀드에서 종종 이뤄지는 의사결정과 동일하다.

In [26]:
def manager_agent(tech_result, fund_result, sent_result, fore_result):
    """
    OpenAI GPT API를 사용하여 4개의 개별 분석 결과를 종합하고
    최종 투자 결정을 내려주는 함수.
    
    :param tech_result: 기술적 분석 결과 
    :param fund_result: 펀더멘털 분석 결과 
    :param sent_result: 감성 분석 결과 
    :param fore_result: 시장 예측 결과
    :return: 최종 투자 판단 (Bullish, Bearish, Neutral 및 매매 조언)
    """
    
    llm = ChatOpenAI(
        model = "gpt-4o-mini", 
        temperature = 0.5, 
        openai_api_key = api_key
    )
    
    # LLM 프롬프트 템플릿 설정
    template = """
    다음은 특정 주식에 대한 4가지 분석 결과입니다.
    각 분석은 해당 주식의 시장 상황을 반영하며, 이를 종합하여 최종 투자 결정을 내려야 합니다.

    - 기술적 분석 (Technical Analysis): {tech}
    - 펀더멘털 분석 (Fundamental Analysis): {fund}
    - 감성 분석 (Sentiment Analysis): {sent}
    - 시장 예측 (Forecasting Analysis): {fore}

    **판단 기준:**
    - Bullish가 3개 이상이면 최종적으로 Bullish (강세)로 판단.
    - Bearish가 3개 이상이면 최종적으로 Bearish (약세)로 판단.
    - 신호가 혼재되어 있거나 Neutral이 많은 경우 최종적으로 Neutral (중립)로 판단.
    - 각 분석 결과의 신뢰도를 고려하여 투자자에게 Buy(매수), Sell(매도), Hold(관망) 조언을 제공.

    현재 상태를 Bullish, Bearish, Neutral 중 하나로 판단한 후, 최종 판단한 상태의 확률(신뢰도)를 퍼센테이지로 제공한 뒤,
    투자자에게 적절한 조언을 한 줄로 제공하세요.
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["tech", "fund", "sent", "fore"]
    )

    # RunnableSequence를 사용하여 체인 구성
    chain = RunnableSequence(
        {
            "tech": RunnablePassthrough(),
            "fund": RunnablePassthrough(),
            "sent": RunnablePassthrough(),
            "fore": RunnablePassthrough()
        }
        | prompt
        | llm
        | (lambda x: x.content)
    )

    # LLM 실행
    result = chain.invoke({
        "tech": tech_result,
        "fund": fund_result,
        "sent": sent_result,
        "fore": fore_result
    })

    return result

In [27]:
final_resp = manager_agent(
    tech_resp,
    fund_resp,
    sent_resp,
    forecast_resp
)

In [28]:
print(final_resp)

현재 상태는 Bearish (약세)로 판단됩니다. 

- 기술적 분석: Bearish 4개
- 펀더멘털 분석: Bearish 4개
- 감성 분석: Bearish 4개
- 시장 예측: Bearish 4개

Bearish가 3개 이상이므로 최종적으로 Bearish로 판단합니다. 

신뢰도: 100%

투자자에게는 "Tesla 주식은 현재 하락세에 있으므로 매도하는 것이 좋습니다."라고 조언합니다.
